In [4]:
import pandas as pd
import numpy as np
#import talib as ta
import os

### Download Kaggle datasets

In [5]:
!kaggle datasets download -d dgawlik/nyse -p ./datasets

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
fundamentals = pd.read_csv("./data/nyse/fundamentals.csv", parse_dates=['Period Ending'])
prices = pd.read_csv("./data/nyse/prices.csv", parse_dates=['date'])
prices_adj = pd.read_csv("./data/nyse/prices-split-adjusted.csv", parse_dates=['date'])
securities = pd.read_csv("./data/nyse/securities.csv", parse_dates=['Date first added'])

In [8]:
symbols = sorted(prices_adj['symbol'].unique())
prices_adj.set_index(['symbol', 'date'], inplace=True, drop=True)
prices_adj.sort_index(inplace=True)

### Normalize the prices entries using the t-1 close price  
Effectively it's the daily return calculated as the difference of the log(price)

In [9]:
log_ret = prices_adj.copy()

# Log transfrom the prices   
for col in ['open','close','low','high']:
    log_ret[col] = np.log(log_ret[col])
    
# Get the close price of the previous day
log_ret['close_m1'] = log_ret['close'].shift(1)

# Calculate normalize the prices using t-1 close price
log_ret.eval('''
                open = open - close_m1
                close = close - close_m1
                high = high - close_m1
                low = low - close_m1
             ''',
             inplace = True)

# Drop close_m1
log_ret.drop(labels=['close_m1'], axis='columns', inplace=True)

# Drop the first entry for each ticker
idx = log_ret.groupby('symbol').head(1).index
log_ret.drop(labels=idx, axis='index', inplace=True)

In [10]:
log_ret.groupby('symbol').head(1).head(6)

,,open,close,low,high,volume
symbol,date,,,,,
A,2010-01-05,-0.002880,-0.010922,-0.017403,-0.002559,4186000.0
AAL,2010-01-05,0.004184,0.107246,-0.012658,0.118482,25212000.0
AAP,2010-01-05,-0.001983,-0.005961,-0.016480,-0.001735,1932400.0
AAPL,2010-01-05,0.002753,0.001727,-0.003558,0.007356,150476200.0
ABBV,2013-01-03,-0.003423,-0.008292,-0.027715,-0.003423,16739300.0
ABC,2010-01-05,0.000751,-0.007160,-0.009432,0.000751,2476800.0


### Some technical analysis indicators

In [219]:
from talib.abstract import STOCH

In [262]:
for s in symbols[:5]:
    
    """ Momentum using close prices """
    prices_adj.loc[[s], 'MOM_20'] = talib.MOM(prices_adj.loc[[s], 'close'], timeperiod=20)
    
    """ Simple moving average of the close prices """
    prices_adj.loc[[s], 'MA_5'] = talib.SMA(prices_adj.loc[[s], 'close'], timeperiod=5)
    prices_adj.loc[[s], 'MA_10'] = talib.SMA(prices_adj.loc[[s], 'close'], timeperiod=10)
    prices_adj.loc[[s], 'MA_20'] = talib.SMA(prices_adj.loc[[s], 'close'], timeperiod=20)
    prices_adj.loc[[s], 'MA_50'] = talib.SMA(prices_adj.loc[[s], 'close'], timeperiod=50)
    prices_adj.loc[[s], 'MA_100'] = talib.SMA(prices_adj.loc[[s], 'close'], timeperiod=100)
    
    """ Bollinger bands """
    upper, middle, lower = talib.BBANDS(prices_adj.loc[[s], 'close'], matype=talib.MA_Type.T3)
    prices_adj.loc[[s], 'BBANDS.upper'] = upper
    prices_adj.loc[[s], 'BBANDS.middle'] = middle
    prices_adj.loc[[s], 'BBANDS.lower'] = lower
    
    """ Stochastic """
    # uses high, low, close (default)
    prices_adj.loc[[s], ['STOCH.slowk','STOCH.slowd']] = STOCH(prices_adj.loc[s], 5, 3, 0, 3, 0).as_matrix()

    # uses high, low, open instead
#     prices_adj.loc[[s], ['STOCH.slowk','STOCH.slowd']] = STOCH(prices_adj.loc[s], 5, 3, 0, 3, 0, 
#                                                                prices=['high', 'low', 'open']).as_matrix()

    

In [21]:
## Calculate return

log_ret = pd.DataFrame()
log_ret.index = prices_adj.index

for s in symbols[:5]:
    prices_adj.loc[[s], 'close_t-1'] = prices_adj.loc[[s], 'close'].shift(1)
    
    log_ret.loc[s, ]

In [22]:
prices_adj.loc['AAL']

,open,close,low,high,volume,close_t-1
date,,,,,,
2010-01-04,4.840000,4.770000,4.660000,4.940000,9837300.0,NaN
2010-01-05,4.790000,5.310000,4.710000,5.370000,25212000.0,4.770000
2010-01-06,5.190000,5.090000,5.000000,5.380000,16597900.0,5.310000
2010-01-07,5.060000,5.240000,5.050000,5.430000,14033400.0,5.090000
2010-01-08,5.270000,5.140000,5.060000,5.430000,12760000.0,5.240000
2010-01-11,5.130000,5.040000,4.940000,5.230000,10952900.0,5.140000
2010-01-12,5.060000,5.080000,4.960000,5.150000,7870300.0,5.040000
2010-01-13,5.120000,5.480000,5.020000,5.500000,16400500.0,5.080000
2010-01-14,5.460000,5.590000,5.410000,5.710000,12767100.0,5.480000


In [ ]:
pd.as_matrix()

In [144]:
# list of functions
print(talib.get_functions())

# dict of functions by group
# print(talib.get_function_groups())

['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR',

In [134]:
abstract.SMA(X, timeperiod=25)

date
2016-01-05           NaN
2016-01-06           NaN
2016-01-07           NaN
2016-01-08           NaN
2016-01-11           NaN
2016-01-12           NaN
2016-01-13           NaN
2016-01-14           NaN
2016-01-15           NaN
2016-01-19           NaN
2016-01-20           NaN
2016-01-21           NaN
2016-01-22           NaN
2016-01-25           NaN
2016-01-26           NaN
2016-01-27           NaN
2016-01-28           NaN
2016-01-29           NaN
2016-02-01           NaN
2016-02-02           NaN
2016-02-03           NaN
2016-02-04           NaN
2016-02-05           NaN
2016-02-08           NaN
2016-02-09    113.553200
2016-02-10    112.820400
2016-02-11    112.306400
2016-02-12    112.022000
2016-02-16    111.788000
2016-02-17    111.638799
                 ...    
2016-11-17    124.103999
2016-11-18    123.962000
2016-11-21    123.834000
2016-11-22    123.706400
2016-11-23    123.538400
2016-11-25    123.451200
2016-11-28    123.354400
2016-11-29    123.238000
2016-11-30    123.15

In [123]:
talib.MA_Type.T3

8

In [124]:
?talib.BBANDS

In [117]:
upper, middle, lower = talib.BBANDS(prices_adj['close'], matype=talib.MA_Type.T3)

In [120]:
lower

symbol  date      
WLTW    2016-01-05           NaN
        2016-01-06           NaN
        2016-01-07           NaN
        2016-01-08           NaN
        2016-01-11           NaN
        2016-01-12           NaN
        2016-01-13           NaN
        2016-01-14           NaN
        2016-01-15           NaN
        2016-01-19           NaN
        2016-01-20           NaN
        2016-01-21           NaN
        2016-01-22           NaN
        2016-01-25           NaN
        2016-01-26           NaN
        2016-01-27           NaN
        2016-01-28           NaN
        2016-01-29           NaN
        2016-02-01           NaN
        2016-02-02           NaN
        2016-02-03           NaN
        2016-02-04           NaN
        2016-02-05           NaN
        2016-02-08           NaN
        2016-02-09    109.446013
        2016-02-10    106.603599
        2016-02-11    105.887765
        2016-02-12    106.354229
        2016-02-16    105.924326
        2016-02-17    10

In [67]:
output = talib.MOM(prices_adj['close'], timeperiod=5)

In [72]:
sum(output.isnull())

5